In [1]:
from kiteconnect import KiteConnect
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os 
import time
from time import sleep 
from pyotp import TOTP
import pyotp
import base64
import binascii
from datetime import datetime
from urllib.parse import urlparse,parse_qs


In [2]:
cwd = os.chdir(r'C:\Users\rsurs\OneDrive\Documents\Zerodha\api_keys')
#print(f"current dir: {os.getcwd()}")


In [3]:
token_path = 'credentials.txt'
key_secret = open(token_path,'r').read().split()
# for i,val in enumerate(key_secret):
#     print(f"{i} : {val}")

In [4]:
api_key = key_secret[0]
api_secret = key_secret[1]
user_name = key_secret[2]
pwd = key_secret[3]
totp_key = key_secret[-1]

# Automate login and extract the request token

In [10]:
kite = KiteConnect(api_key=api_key)
service = Service(ChromeDriverManager().install())
service.start()
options = Options()
#options.add_argument('--headless')
options.to_capabilities()
driver = webdriver.Remote(
    command_executor=service.service_url,
    options=options)
driver.get(kite.login_url())
driver.implicitly_wait(5)
username = driver.find_element(By.XPATH, "//input[@type='text']")
username.send_keys(user_name)
# print(username)
password = driver.find_element(By.XPATH, "//input[@type='password']")
password.send_keys(pwd)
# print(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
sleep(1)
totp = driver.find_element(By.XPATH,"//input[@type='number']")
totp_token = TOTP(totp_key)
#print(totp_token)
token = totp_token.now()
# print(token)
totp.send_keys(token)
driver.find_element(By.XPATH,"//button[@type = 'submit']").click()
sleep(1)
current_url = driver.current_url
# print(current_url)
parsed_url = urlparse(current_url)
# print(parsed_url)
query_params = parse_qs(parsed_url.query)
# print(query_params) 
request_token = query_params.get('request_token',[None])[0]
# print(request_token)
with open('request_token.txt', 'w') as f:
    f.write(request_token)
# print(r)
driver.quit()

# Generating the access token
Note the access token is valid until 6 am

In [ ]:
request_token = open('request_token.txt','r').read()
kite = KiteConnect(api_key=api_key)
data = kite.generate_session(request_token=request_token,api_secret=api_secret)
data
with open('access_token.txt','w') as f:
    f.write(data['access_token'])
access_token = open('access_token.txt','r').read()
print(access_token)     

